# 20기 KNN 정규세션 과제

**데이터:** [blackfriday | Kaggle](https://www.kaggle.com/llopesolivei/blackfriday)

---

## 0. 데이터 불러오기

In [68]:
origin_df = pd.read_csv("blackfriday.csv", index_col = 0)

In [69]:
import pandas as pd
origin_df = pd.read_csv("blackfriday.csv", index_col = 0)
df = origin_df.copy()
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,NaN,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,NaN,NaN,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,NaN,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,NaN,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,NaN,5780


## 1. Preprocssing / EDA

### 기본데이터 확ㅇ니

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     4998 non-null   int64  
 1   Product_ID                  4998 non-null   object 
 2   Gender                      4998 non-null   object 
 3   Age                         4998 non-null   object 
 4   Occupation                  4998 non-null   int64  
 5   City_Category               4998 non-null   object 
 6   Stay_In_Current_City_Years  4998 non-null   object 
 7   Marital_Status              4998 non-null   int64  
 8   Product_Category_1          4998 non-null   int64  
 9   Product_Category_2          3465 non-null   float64
 10  Product_Category_3          1544 non-null   float64
 11  Purchase                    4998 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 507.6+ KB


### 결측치 확인

In [71]:
df.isna().sum()

User_ID                          0
Product_ID                       0
Gender                           0
Age                              0
Occupation                       0
City_Category                    0
Stay_In_Current_City_Years       0
Marital_Status                   0
Product_Category_1               0
Product_Category_2            1533
Product_Category_3            3454
Purchase                         0
dtype: int64

### 결측치 채우기

In [72]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
df[['Product_Category_2','Product_Category_3']] = imputer.fit_transform(df[['Product_Category_2','Product_Category_3']])

### 필요한 column 만 선택


In [73]:
cols=df.columns
obj_cols=df.select_dtypes(include='object').columns
num_cols=df.select_dtypes(exclude='object').columns
(num_cols[3:])

Index(['Product_Category_1', 'Product_Category_2', 'Product_Category_3',
       'Purchase'],
      dtype='object')

### train test split

In [74]:
# train ttest split df
# import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[num_cols[3:]], df['Marital_Status'], test_size=0.2, random_state=42)

### scaling

In [75]:
# scale X_train
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)



In [76]:
X_test_scaled= scaler.transform(X_test)

## 2. KNN 구현 & 파라미터 튜닝

In [83]:
# knn clustering with df
from sklearn.neighbors import KNeighborsClassifier
#import gridsearchcv
from sklearn.model_selection import GridSearchCV
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled,y_train)
grid_params = {
    'n_neighbors' : list(range(1,20)),
    'weights' : ["uniform", "distance"],
    'metric' : ['euclidean', 'manhattan', 'minkowski']
}
gs_s = GridSearchCV(knn, grid_params, cv=10)
gs_s.fit(X_train_scaled, y_train)
print("Best Parameters : ", gs_s.best_params_)
print("Best Score : ", gs_s.best_score_)
print("Best Test Score : ", gs_s.score(X_test_scaled, y_test))

Best Parameters :  {'metric': 'euclidean', 'n_neighbors': 6, 'weights': 'uniform'}
Best Score :  0.5555281954887219
Best Test Score :  0.53


## 3. Evaluation

In [ ]:
import f1socre
from sklearn.metrics import f1_score
y_pred = gs_s.predict(X_test)
f1_score(y_test, y_pred, average='macro')